In [ ]:
# --- Importaciones de Bibliotecas ---
import re  # Para operaciones con expresiones regulares (limpieza de texto, extracción)
import os  # Para interactuar con el sistema operativo (manejar rutas, nombres de archivo)
from chromadb import PersistentClient # Cliente para interactuar con la base de datos ChromaDB persistente
import fitz  # Biblioteca PyMuPDF para trabajar con archivos PDF (extraer texto)
from langchain.text_splitter import RecursiveCharacterTextSplitter # Para dividir texto en fragmentos (chunks)
from langchain.schema import Document # Estructura de datos para representar documentos en LangChain
from langchain_chroma import Chroma # Integración de LangChain con ChromaDB para crear vector stores
from langchain_openai import OpenAIEmbeddings # Para generar embeddings usando modelos de OpenAI
from dotenv import load_dotenv # Para cargar variables de entorno desde un archivo .env

# Carga las variables de entorno (por ejemplo, la API key de OpenAI)
load_dotenv()

# --- Configuración y Constantes ---

# Lista de diccionarios que define los archivos PDF a procesar.
# Cada diccionario especifica la 'ruta' del PDF y opcionalmente 'fechas' para filtrar artículos.
PDFS_A_PROCESAR = [
    {
        "ruta": "Ley Actualizada 2025 .pdf", # Ruta al primer archivo PDF
        "fechas": None # No se aplica filtro de fecha para este archivo
    },
    {
        "ruta": "Ley del 2024 .pdf", # Ruta al segundo archivo PDF
        "fechas": None # No se aplica filtro de fecha para este archivo
    },
    # Se pueden añadir más diccionarios para procesar otros PDFs
]

# Tamaño máximo de cada fragmento de texto al dividir.
CHUNK_SIZE = 1000
# Número de caracteres que se solaparán entre fragmentos consecutivos.
CHUNK_OVERLAP = 100
# Directorio donde se guardará la base de datos vectorial ChromaDB.
VECTORSTORE_DIR = "./chroma_db"
# Identificador del modelo de embeddings a utilizar (actualmente OpenAI).
CURRENT_MODEL = "openai"
# Nombre de la colección dentro de ChromaDB donde se almacenarán los vectores.
COLLECTION_NAME = "leyes_mexico_selectivo"

# --- Funciones de Procesamiento ---

def limpiar_texto(texto):
    """
    Elimina espacios en blanco redundantes, normaliza saltos de línea
    y corrige un patrón específico en los nombres de artículos.

    Args:
        texto (str): El texto a limpiar.

    Returns:
        str: El texto limpio.
    """
    texto = re.sub(r'\s+', ' ', texto) # Reemplaza múltiples espacios/saltos con uno solo
    texto = re.sub(r'\n\s*\n', '\n\n', texto) # Normaliza saltos de línea dobles
    texto = re.sub(r'(Artículo\s+\d+)o(?=[.-])', r'\1', texto) # Corrige "Artículo Xo." a "Artículo X."
    return texto.strip() # Elimina espacios al inicio y final

def dividir_texto(texto, chunk_size=1000, chunk_overlap=100):
    """
    Divide un texto largo en fragmentos más pequeños con solapamiento.

    Args:
        texto (str): El texto a dividir.
        chunk_size (int): Tamaño máximo de cada fragmento.
        chunk_overlap (int): Solapamiento entre fragmentos.

    Returns:
        list[str]: Una lista de fragmentos de texto.
    """
    # Inicializa el divisor de texto recursivo con los parámetros especificados.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    # Realiza la división del texto.
    return text_splitter.split_text(texto)

def extraer_texto_sin_encabezado_pie(ruta_pdf, margen_superior=10, margen_inferior=10):
    """
    Extrae el texto de un archivo PDF, intentando omitir encabezados y pies de página
    basándose en márgenes porcentuales superior e inferior.

    Args:
        ruta_pdf (str): La ruta al archivo PDF.
        margen_superior (int): Porcentaje superior de la página a ignorar.
        margen_inferior (int): Porcentaje inferior de la página a ignorar.

    Returns:
        str: El texto extraído y concatenado de todas las páginas, o una cadena vacía si hay error.
    """
    texto_completo = ""
    try:
        # Abre el documento PDF.
        documento = fitz.open(ruta_pdf)
        print(f"Procesando archivo: {ruta_pdf}")
        # Itera sobre cada página del documento.
        for i, pagina in enumerate(documento):
            # Obtiene las dimensiones de la página.
            altura = pagina.rect.height
            anchura = pagina.rect.width
            # Define el rectángulo del área de contenido principal excluyendo márgenes.
            rect_contenido = fitz.Rect(
                0,
                altura * margen_superior / 100.0, # Coordenada Y superior
                anchura,
                altura * (1 - margen_inferior / 100.0) # Coordenada Y inferior
            )
            # Extrae el texto únicamente dentro del rectángulo definido.
            texto_pagina = pagina.get_text("text", clip=rect_contenido)
            # Limpia el texto extraído de la página.
            texto_limpio_pagina = limpiar_texto(texto_pagina)
            # Si la página contiene texto después de la limpieza, lo añade al resultado total.
            if texto_limpio_pagina:
                texto_completo += texto_limpio_pagina + "\n\n" # Añade separación entre páginas

        # Cierra el documento PDF.
        documento.close()
    except Exception as e:
        # Maneja posibles errores durante la apertura o procesamiento del PDF.
        print(f"Error al procesar el archivo {ruta_pdf}: {e}")
        return "" # Devuelve cadena vacía en caso de error.
    # Devuelve el texto completo extraído.
    return texto_completo

def extraer_articulos_para_rag(texto_completo, nombre_archivo, fechas=None, chunk_size=1000, chunk_overlap=100):
    """
    Identifica artículos en el texto, los filtra opcionalmente por fecha,
    los divide en fragmentos (chunks) y les asocia metadatos relevantes.

    Args:
        texto_completo (str): El texto completo del documento de donde extraer artículos.
        nombre_archivo (str): El nombre del archivo PDF original para usar en metadatos.
        fechas (list[str], optional): Lista de fechas (como cadenas) para filtrar artículos.
                                       Si es None o vacía, no se aplica filtro. Defaults to None.
        chunk_size (int): Tamaño máximo de cada fragmento.
        chunk_overlap (int): Solapamiento entre fragmentos.

    Returns:
        list[dict]: Una lista de diccionarios, donde cada diccionario contiene
                    el 'texto' del fragmento y sus 'metadatos'.
    """
    # Expresión regular para encontrar bloques de texto que corresponden a artículos.
    # Busca "Artículo [número]" seguido de "." o "-", captura el encabezado y el contenido
    # hasta el siguiente artículo o el final del texto.
    articulos = re.findall(r"(Artículo \d+.*?)\s*(-|\.)\s*(.*?)(?=Artículo \d+.*?[-.]|$)", texto_completo, re.DOTALL)
    resultados = []
    # Procesa las coincidencias para separar encabezado y contenido.
    for articulo in articulos:
        encabezado = articulo[0].strip() + articulo[1].strip() # Combina número y separador
        contenido = articulo[2].strip() # Contenido del artículo
        resultados.append((encabezado, contenido))
    fragmentos_con_metadatos = [] # Lista para almacenar los chunks finales con metadatos

    # Asegura que 'fechas' sea None si la lista está vacía para simplificar la lógica.
    if isinstance(fechas, list) and not fechas:
        fechas = None

    # Informa si se aplicará filtro de fecha.
    if fechas:
        print(f"  Buscando artículos en '{nombre_archivo}' que contengan las fechas: {', '.join(fechas)}")
    else:
        print(f"  Extrayendo todos los artículos de '{nombre_archivo}' (sin filtro de fecha).")


    articulos_encontrados_en_archivo = 0 # Contador de artículos procesados
    # Itera sobre cada artículo encontrado (encabezado, contenido).
    for encabezado, contenido in resultados:
        procesar_articulo = True # Bandera para decidir si procesar el artículo actual

        # Aplica el filtro de fecha si se proporcionaron fechas.
        if fechas:
            fecha_encontrada = False
            # Comprueba si alguna de las fechas buscadas está en el contenido del artículo.
            for fecha in fechas:
                if re.search(re.escape(fecha), contenido, re.IGNORECASE): # Búsqueda insensible a mayúsculas
                    fecha_encontrada = True
                    break # Si se encuentra una fecha, no es necesario seguir buscando en este artículo
            # Si no se encontró ninguna de las fechas requeridas, no se procesa el artículo.
            if not fecha_encontrada:
                procesar_articulo = False

        # Si el artículo pasa el filtro (o no hay filtro), procede a dividirlo y añadir metadatos.
        if procesar_articulo:
            # Intenta extraer el número del artículo del encabezado.
            match_numero = re.search(r'\d+', encabezado)
            numero_articulo = int(match_numero.group(0)) if match_numero else 0 # 0 si no se encuentra número

            articulos_encontrados_en_archivo += 1
            # Obtiene el nombre del archivo sin la extensión para usar como fuente.
            nombre_archivo_sin_extension = os.path.splitext(nombre_archivo)[0]

            # Combina el nombre del archivo (como contexto), encabezado y contenido para dividir.
            texto_articulo = nombre_archivo_sin_extension + " " + encabezado + " " + contenido
            # Divide el texto del artículo en fragmentos.
            chunks = dividir_texto(texto_articulo, chunk_size, chunk_overlap)

            # Itera sobre cada fragmento (chunk) del artículo.
            for i, chunk in enumerate(chunks):
                # Añade el encabezado al inicio de los chunks > 0 si no lo tienen ya, para dar contexto.
                if i > 0 and not chunk.strip().lower().startswith(encabezado.strip().lower()):
                    # Formatea el chunk con el encabezado y una marca de continuación.
                    chunk_con_contexto = f"{encabezado.strip()} (continuación)... {chunk.strip()}"
                else:
                    # Usa el chunk tal cual si es el primero o ya incluye el encabezado.
                    chunk_con_contexto = chunk

                # Crea el diccionario de metadatos para el fragmento actual.
                metadatos = {
                    "titulo": encabezado.strip(), # El encabezado completo del artículo
                    "numero_articulo": numero_articulo, # El número extraído del artículo
                    "fuente": nombre_archivo_sin_extension, # Nombre del archivo PDF origen
                    "parte": i + 1, # Número de fragmento dentro del artículo
                    "total_partes": len(chunks) # Total de fragmentos para este artículo
                }
                # Añade el fragmento (texto con contexto) y sus metadatos a la lista.
                fragmentos_con_metadatos.append({"texto": chunk_con_contexto, "metadatos": metadatos})

    # Informa cuántos artículos relevantes se procesaron del archivo actual.
    print(f"  Se procesaron {articulos_encontrados_en_archivo} artículos relevantes en '{nombre_archivo_sin_extension}'.")
    # Devuelve la lista de fragmentos con metadatos.
    return fragmentos_con_metadatos

def crear_documentos(fragmentos_con_metadatos):
    """
    Convierte una lista de diccionarios (fragmento y metadatos) en una lista
    de objetos Document de LangChain.

    Args:
        fragmentos_con_metadatos (list[dict]): Lista de diccionarios con 'texto' y 'metadatos'.

    Returns:
        list[Document]: Lista de objetos Document listos para LangChain.
    """
    # Utiliza una comprensión de lista para crear un objeto Document por cada fragmento.
    return [Document(page_content=frag["texto"], metadata=frag["metadatos"])
            for frag in fragmentos_con_metadatos]

def get_embeddings():
    """
    Obtiene e inicializa el modelo de embeddings configurado en CURRENT_MODEL.

    Raises:
        ValueError: Si CURRENT_MODEL no es un valor soportado.

    Returns:
        Embeddings: Una instancia del modelo de embeddings de LangChain.
    """
    global CURRENT_MODEL # Accede a la variable global
    if CURRENT_MODEL == "openai":
        # Inicializa y devuelve el modelo de embeddings de OpenAI.
        return OpenAIEmbeddings(model="text-embedding-3-small")
    else:
        # Lanza un error si el modelo configurado no es reconocido.
        raise ValueError(f"Modelo de embeddings no soportado: {CURRENT_MODEL}")

def crear_vectorstore(documentos, persist_directory, collection_name):
    """
    Crea (o actualiza) una base de datos vectorial ChromaDB con los documentos
    proporcionados y la guarda de forma persistente.

    Args:
        documentos (list[Document]): Lista de objetos Document de LangChain.
        persist_directory (str): Ruta del directorio donde guardar/cargar la BD.
        collection_name (str): Nombre de la colección dentro de ChromaDB.

    Returns:
        Chroma: La instancia del vectorstore creado, o None si hubo un error o no hay documentos.
    """
    # Verifica si hay documentos para procesar.
    if not documentos:
        print("No hay documentos para añadir al vectorstore.")
        return None

    try:
        # Obtiene el modelo de embeddings configurado.
        embeddings = get_embeddings()
        print(f"Creando/actualizando vectorstore en '{persist_directory}' con la colección '{collection_name}'...")
        # Crea el vectorstore usando Chroma.from_documents.
        # Esto calcula los embeddings para cada documento y los almacena.
        vectorstore = Chroma.from_documents(
            documents=documentos, # Los documentos a añadir
            embedding=embeddings, # El modelo para generar embeddings
            persist_directory=persist_directory, # Directorio para persistencia
            collection_name=collection_name # Nombre de la colección
        )
        print("Vectorstore creado/actualizado exitosamente.")
        # Devuelve la instancia del vectorstore.
        return vectorstore
    except Exception as e:
        # Captura y muestra errores durante la creación del vectorstore.
        print(f"Error al crear el vectorstore: {e}")
        # Devuelve None para indicar el fallo.
        return None

def delete_collection(collection_name, persist_directory):
    """
    Elimina una colección específica de una base de datos ChromaDB persistente.

    Args:
        collection_name (str): El nombre de la colección a eliminar.
        persist_directory (str): La ruta al directorio de la base de datos persistente.
    """
    try:
        print(f"Intentando eliminar la colección '{collection_name}' de '{persist_directory}'...")
        # Crea un cliente Chroma que apunta al directorio persistente.
        chroma_client = PersistentClient(path=persist_directory)
        # Intenta eliminar la colección por su nombre.
        chroma_client.delete_collection(collection_name)
        print(f"Colección '{collection_name}' eliminada exitosamente.")
    except Exception as e:
        # Captura cualquier error, como que la colección no exista.
        print(f"No se pudo eliminar la colección '{collection_name}': {e}")

# --- Función Principal de Ejecución ---

def main():
    """
    Orquesta el flujo completo del script:
    1. (Opcional) Elimina la colección existente.
    2. Itera sobre los PDFs configurados.
    3. Para cada PDF: extrae texto, extrae/filtra artículos, los divide en fragmentos.
    4. Agrupa todos los fragmentos.
    5. Convierte fragmentos a Documentos LangChain.
    6. Crea/actualiza el vectorstore ChromaDB con los documentos.
    """
    # Opcional: Elimina la colección existente para empezar desde cero.
    # Comenta esta línea si deseas añadir documentos a una colección existente.
    delete_collection(COLLECTION_NAME, VECTORSTORE_DIR)

    todos_los_fragmentos_rag = [] # Lista para acumular fragmentos de todos los PDFs

    # Itera sobre la configuración de cada PDF definido en PDFS_A_PROCESAR.
    for config_pdf in PDFS_A_PROCESAR:
        ruta_pdf = config_pdf.get("ruta") # Obtiene la ruta del archivo
        fechas_filtro = config_pdf.get("fechas") # Obtiene las fechas para filtrar (puede ser None)

        # Verifica si la ruta está definida en la configuración.
        if not ruta_pdf:
            print("Advertencia: Se encontró una configuración sin 'ruta'. Saltando...")
            continue # Pasa a la siguiente configuración

        # Verifica si el archivo PDF existe en la ruta especificada.
        if not os.path.exists(ruta_pdf):
             print(f"Advertencia: El archivo '{ruta_pdf}' no se encontró. Saltando...")
             continue # Pasa al siguiente archivo si no existe

        # 1. Extrae el texto del PDF actual, omitiendo encabezados/pies.
        texto_completo_pdf = extraer_texto_sin_encabezado_pie(ruta_pdf)
        # Si no se pudo extraer texto o el archivo estaba vacío, pasa al siguiente.
        if not texto_completo_pdf:
            print(f"No se pudo extraer texto de '{ruta_pdf}' o está vacío. Saltando...")
            continue # Pasa al siguiente archivo

        # 2. Extrae artículos, filtra por fecha (si aplica) y divide en fragmentos con metadatos.
        fragmentos_pdf = extraer_articulos_para_rag(
            texto_completo_pdf,
            nombre_archivo=os.path.basename(ruta_pdf), # Usa solo el nombre base del archivo
            fechas=fechas_filtro, # Pasa las fechas de filtro específicas para este PDF
            chunk_size=CHUNK_SIZE, # Usa las constantes globales
            chunk_overlap=CHUNK_OVERLAP # Usa las constantes globales
        )

        # 3. Añade los fragmentos obtenidos de este PDF a la lista general.
        todos_los_fragmentos_rag.extend(fragmentos_pdf)
        print(f"Se añadieron {len(fragmentos_pdf)} fragmentos del archivo '{ruta_pdf}'.\n")

    # Si después de procesar todos los PDFs no se acumuló ningún fragmento, termina.
    if not todos_los_fragmentos_rag:
        print("No se encontraron fragmentos relevantes en ninguno de los archivos PDF procesados.")
        return # Finaliza la ejecución

    # Informa el total de fragmentos recopilados.
    print(f"\nTotal de fragmentos recopilados de todos los archivos: {len(todos_los_fragmentos_rag)}")

    # 4. Convierte la lista de diccionarios de fragmentos a objetos Document de LangChain.
    documentos_langchain = crear_documentos(todos_los_fragmentos_rag)
    print(f"Se crearon {len(documentos_langchain)} documentos LangChain.")

    # 5. Crea o actualiza el vectorstore persistente con los documentos LangChain.
    vectorstore = crear_vectorstore(documentos_langchain, VECTORSTORE_DIR, collection_name=COLLECTION_NAME)

    # Informa el resultado final del proceso.
    if vectorstore:
        print(f"\nProceso completado. Vectorstore ('{COLLECTION_NAME}') guardado/actualizado en '{VECTORSTORE_DIR}'.")
    else:
        print("\nEl proceso finalizó, pero hubo un error al crear el vectorstore.")

# --- Punto de Entrada ---
# Asegura que la función main() se ejecute solo cuando el script se corre directamente.
if __name__ == "__main__":
    main()


Intentando eliminar la colección 'leyes_mexico_selectivo' de './chroma_db'...
Colección 'leyes_mexico_selectivo' eliminada exitosamente.
Procesando archivo: Ley Actualizada 2025 .pdf
  Extrayendo todos los artículos de 'Ley Actualizada 2025 .pdf' (sin filtro de fecha).
  Se procesaron 40 artículos relevantes en 'Ley Actualizada 2025 '.
Se añadieron 90 fragmentos del archivo 'Ley Actualizada 2025 .pdf'.

Procesando archivo: Ley del 2024 .pdf
  Extrayendo todos los artículos de 'Ley del 2024 .pdf' (sin filtro de fecha).
  Se procesaron 1318 artículos relevantes en 'Ley del 2024 '.
Se añadieron 2284 fragmentos del archivo 'Ley del 2024 .pdf'.


Total de fragmentos recopilados de todos los archivos: 2374
Se crearon 2374 documentos LangChain.
Creando/actualizando vectorstore en './chroma_db' con la colección 'leyes_mexico_selectivo'...
Vectorstore creado/actualizado exitosamente.

Proceso completado. Vectorstore ('leyes_mexico_selectivo') guardado/actualizado en './chroma_db'.


In [ ]:
# Importar las bibliotecas necesarias
import pprint # Para imprimir diccionarios y listas de forma más legible

# --- Configuración de Constantes ---
# Define el directorio donde se almacenará la base de datos vectorial persistente
VECTORSTORE_DIR = "./chroma_db"
# Define el nombre de la colección dentro de ChromaDB a la que nos conectaremos
COLLECTION_NAME = "leyes_mexico_selectivo"

# --- Inicialización del Cliente ChromaDB ---
# Establece la ruta donde se guardarán los datos de ChromaDB
ruta_persistencia = VECTORSTORE_DIR
# Crea un cliente persistente de ChromaDB que guarda los datos en el disco en la ruta especificada
cliente = chromadb.PersistentClient(path=ruta_persistencia)

# Asigna el nombre de la colección a una variable para fácil acceso
nombre_coleccion = COLLECTION_NAME

# --- Bloque Principal de Ejecución (Manejo de Errores) ---
try:
    # Intenta obtener (acceder a) la colección especificada por su nombre
    print(f"Intentando acceder a la colección: '{nombre_coleccion}'...")
    coleccion = cliente.get_collection(name=nombre_coleccion)
    print(f"Acceso exitoso a la colección '{nombre_coleccion}'.")

    # Obtiene y muestra el número total de ítems (documentos/vectores) en la colección
    count = coleccion.count()
    print(f"La colección tiene {count} ítems.")

    # --- Recuperación de Datos ---
    print("\nRecuperando datos (primeros 3000, incluyendo documentos y metadatos)...")
    # Llama al método get() de la colección para recuperar datos
    resultados = coleccion.get(
        limit=3000, # Especifica el número máximo de ítems a recuperar
        include=["metadatas", "documents"] # Indica que queremos recuperar los metadatos y los documentos asociados a cada ítem
        # Se podría añadir "embeddings" a la lista 'include' para obtener también los vectores
    )

    # --- Procesamiento e Impresión de Resultados ---
    print("\n--- Contenido Recuperado ---")
    # Verifica si se obtuvieron resultados y si la lista de IDs no está vacía
    if resultados and resultados.get('ids'):
        # Obtiene el número de resultados recuperados (puede ser menor al límite si hay menos ítems)
        num_resultados = len(resultados['ids'])
        print(f"Mostrando {num_resultados} de {count} ítems:")

        # Itera sobre cada uno de los ítems recuperados
        for i in range(num_resultados):
            print(f"\nÍtem {i+1}:") # Muestra un encabezado para cada ítem
            # Imprime el ID único del ítem
            print(f"  ID: {resultados['ids'][i]}")

            # Verifica si se recuperaron documentos y si el documento actual no es nulo
            if resultados.get('documents') and resultados['documents'][i] is not None:
                # Imprime el contenido del documento. repr() se usa para mostrar caracteres especiales como \n
                print(f"  Documento: {repr(resultados['documents'][i])}")
            else:
                # Mensaje si el documento no está disponible o no se solicitó
                print("  Documento: (No solicitado o no disponible)")

            # Verifica si se recuperaron metadatos y si los metadatos actuales no son nulos
            if resultados.get('metadatas') and resultados['metadatas'][i] is not None:
                # Imprime la etiqueta "Metadatos"
                print( "  Metadatos:")
                # Usa pprint para imprimir el diccionario de metadatos de forma indentada y legible
                pprint.pprint(resultados['metadatas'][i], indent=4)
            else:
                # Mensaje si los metadatos no están disponibles o no se solicitaron
                print("  Metadatos: (No solicitados o no disponibles)")

    else:
        # Mensaje si no se encontraron ítems en la colección o si está vacía
        print("No se encontraron datos en la colección con los criterios dados o la colección está vacía.")

# --- Manejo de Excepciones Generales ---
except Exception as e:
    # Imprime un mensaje de error si ocurre cualquier problema durante la ejecución del bloque try
    print(f"\nError: {e}")
    print("Verifica que el nombre de la colección sea correcto y que el cliente ChromaDB esté configurado adecuadamente.")
    # Intenta listar las colecciones disponibles como ayuda para depuración
    try:
        lista_colecciones = cliente.list_collections()
        print("\nColecciones disponibles:")
        if lista_colecciones:
            # Imprime los nombres de las colecciones encontradas
            for col in lista_colecciones:
                print(f"- {col.name}")
        else:
            # Mensaje si no hay colecciones en la instancia de ChromaDB
            print("(Ninguna)")
    # Manejo de error específico al intentar listar colecciones
    except Exception as list_e:
        print(f"No se pudieron listar las colecciones: {list_e}")


Intentando acceder a la colección: 'leyes_mexico_selectivo'...
Acceso exitoso a la colección 'leyes_mexico_selectivo'.
La colección tiene 2367 ítems.

Recuperando datos (primeros 100, incluyendo documentos y metadatos)...

--- Contenido Recuperado ---
Mostrando 2367 de 2367 ítems:

Ítem 1:
  ID: 81abae26-5daf-4231-9391-7d3969fdedd4
  Documento: 'Artículo 1.- La presente Ley es de observancia general en toda la República y rige las relaciones de trabajo comprendidas en el artículo 123, Apartado A, de la Constitución.'
  Metadatos:
{'numero_articulo': 1, 'parte': 1, 'titulo': 'Artículo 1.', 'total_partes': 1}

Ítem 2:
  ID: 6a661ebb-6080-4b8c-9592-31dce5986ac3
  Documento: 'Artículo 2.- Las normas del trabajo tienden a conseguir el equilibrio entre los factores de la producción y la justicia social, así como propiciar el trabajo digno o decente en todas las relaciones laborales. Se entiende por trabajo digno o decente aquél en el que se respeta plenamente la dignidad humana del trabajado